In [1]:
import os

documents = []

for (root, dirs, files) in os.walk('documents'):
    for file in files:
        fp = f'{root}/{file}'
        with open(fp, 'r', encoding='utf-8') as f:
            content = f.read()
        
        doc = {
            'name': file.replace('.md', ''),
            'content': content
        }

        documents.append(doc)

print(len(documents))

94


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [3]:
char_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ", ", " ", ""],
    chunk_size=1000,
    chunk_overlap=10
)
char_splitter_texts = char_splitter.split_text('\n\n'.join([d['content'] for d in documents]))

print(len(char_splitter_texts))

589


In [4]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=10, tokens_per_chunk=256)

token_splitter_texts = []
for text in char_splitter_texts:
    token_splitter_texts += token_splitter.split_text(text)

print(len(token_splitter_texts))

/home/lisboa/miniconda3/envs/promptneering/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


706


In [5]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()

In [ ]:
client = chromadb.PersistentClient()
collection = client.get_or_create_collection("prompt_engineering_knowledge", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_splitter_texts))]

# collection.add(ids=ids, documents=token_splitter_texts)

print(collection.count())

706


In [14]:
results = collection.query(query_texts=['How to make a few show prompting?'], n_results=10)

In [15]:
results['documents']

[["for example, you might be interested in learning the concept of prompt engineering. you might try something like : ` ` ` explain the concept prompt engineering. keep the explanation short, only a few sentences, and don ' t be too descriptive. ` ` ` it ' s not clear from the prompt above how many sentences to use and what style. you might still somewhat get good responses with the above prompts but the better prompt would be one that is very specific, concise, and to the point. something like : ` ` ` use 2 - 3 sentences to explain the concept of prompt engineering to a high school student. ` ` ` # # # to do or not to do? another common tip when designing prompts is to avoid saying what not to do but say what to do instead. this encourages more specificity and focuses on the details that lead to good responses from the model. here is an example of a movie recommendation chatbot failing at exactly what i don ' t want it to do because of how i wrote the instruction - - focusing on what 